# Build a simple LLM application with chat models and prompt templates

LangChain has implementations for older language models that take a string as input and return a string as output. These models are typically named without the "Chat" prefix (e.g., Ollama, Anthropic, OpenAI, etc.), and may include the "LLM" suffix (e.g., OllamaLLM, AnthropicLLM, OpenAILLM, etc.). These models implement the BaseLLM interface.

Users should be using almost exclusively the newer Chat Models as most model providers have adopted a chat like interface for interacting with language models.

In [1]:
from langchain_ollama import ChatOllama

In [2]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

In [3]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

# Invocation

In [7]:
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Ciao! Come posso aiutarti oggi?', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-04-04T06:10:32.7218029Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2862656900, 'load_duration': 33160300, 'prompt_eval_count': 24, 'prompt_eval_duration': 239000000, 'eval_count': 12, 'eval_duration': 2589000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-4965e76f-5f1d-442a-8743-af167ef5b6cf-0', usage_metadata={'input_tokens': 24, 'output_tokens': 12, 'total_tokens': 36})

In [9]:
print(ai_msg.content)

The translation of "I love programming" in French is:

"J'adore le programmation."


In [6]:
for chunk in llm.stream(messages):
    print(chunk.text(), end = "")

Ciao! Come posso aiutarti oggi?

In [ ]:
# The following are equivalent
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

# Chaining

In [11]:
from langchain_core.prompts import ChatPromptTemplate

In [51]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}. Translate the given sentence.",
        ),
        ("human", "{input}"),
    ]
)

In [52]:
op = prompt.invoke({
    'input_language': "English",
    'output_language': "Hindi",
    'input': 'What are you doing'
})
op.to_string()

'System: You are a helpful assistant that translates English to Hindi. Translate the given sentence.\nHuman: What are you doing'

In [53]:
op.to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to Hindi. Translate the given sentence.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What are you doing', additional_kwargs={}, response_metadata={})]

In [54]:
chain = prompt | llm

In [55]:
ip = {
    'input_language': "English",
    'output_language': "Hindi",
    'input': 'I love programming'
}

In [56]:
for chunk in chain.stream(ip):
    print(chunk.text(), end="")


The translation of "I love programming" in Hindi is:

मैं प्रोग्रामिंग को पसंद करता हूँ।

# Prompt Templates

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [7]:
system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [14]:
prompt = prompt_template.invoke({"language": "Hindi", "text": "I love programming"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming', additional_kwargs={}, response_metadata={})])

In [15]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming', additional_kwargs={}, response_metadata={})]

In [16]:
response = llm.invoke(prompt)
print(response.content)

मैं प्रोग्रामिंग को पसंद करता हूँ।
